<a href="https://colab.research.google.com/github/jhill1/tidal_analysis_workshop/blob/main/Tidal_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Tides are a fundamental part of a number of coastal processes. As sea levels rise, tides will change, and hence our flood risk, coastal geomorphology and ecological systems will also change as a consequence.

This Jupyter notebook will take you through the analysis of sea level data to extract tidal information.


# What to do

Run each block of code sequentially. In some places, template code is given for you to edit to carry out your own analysis.

There are questions posed along the way to help gain understanding.


In [ ]:
# let's install some essential python modules
!pip install wget
!pip install uptide

In [ ]:
# import the modules we need
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import wget
import os
import numpy as np
import uptide
import pytz
import math


Now we have out environment set up. We can create a couple of "helper" functions to make life easier later!

In [ ]:
def read_and_process_data(filename):
    tide_data = pd.read_csv(filename, header=None)
    tide_data['Date'] = pd.to_datetime(dict(year=tide_data[0], month=tide_data[1], day=tide_data[2], hour=tide_data[3]))
    # col 0 is year, col 1 is month, col2 is day, col3 hour
    tide_data = tide_data.drop([0,1,2,3], axis = 1)
    tide_data = tide_data.rename(columns={4: "Tide"})
    tide_data = tide_data.set_index('Date')
    tide_data = tide_data.mask(tide_data['Tide'] < -300)

    return tide_data

def extract_single_year_remove_mean(year, data):
    year_string_start = str(year)+"0101"
    year_string_end = str(year)+"1231"
    year_data = data.loc[year_string_start:year_string_end, ['Tide']]
    # remove mean to oscillate around zero
    mmm = np.mean(year_data['Tide'])
    year_data['Tide'] -= mmm

    return year_data

We are going to use data from the long term tidal record dataset held by the School of Ocean and Earth Science and Technology, University of Hawaii. Some of their tidal records go back over 100 years. We're going to download data for three locations in Australia: Freemantle, WA; Booby Island, QLD; and Fort Denison, NSW.

In [ ]:
FortDenison_url = "https://uhslc.soest.hawaii.edu/data/csv/fast/hourly/h333.csv"
BoobyIsland_url = "https://uhslc.soest.hawaii.edu/data/csv/fast/hourly/h336.csv"
Freemantle_url = "https://uhslc.soest.hawaii.edu/data/csv/fast/hourly/h175.csv"
urls = [FortDenison_url, BoobyIsland_url, Freemantle_url]

# fetch our data and store
for url in urls:
    file_name = os.path.basename(url) # get the full path to the file
    if os.path.exists(file_name):
        os.remove(file_name) # if exists, remove it directly
    file_name = wget.download(url, out=".")

We now have three csv files which should be stored in your temporary files here (click the little folder symbol to the left and you should see the three .csv files).

In [ ]:
# load and store as a pandas dataframe
Fort_Denison = read_and_process_data("h333.csv")
Booby_Island = read_and_process_data("h336.csv")
Freemantle = read_and_process_data("h175.csv")


In [ ]:
# Let's plot 1 years' worth of tidal data
fig_summary=plt.figure()
ax=fig_summary.add_subplot(111)
fd = ax.plot(Fort_Denison['Tide'], color="blue", lw=1, label="Fort Denison")
bi = ax.plot(Booby_Island['Tide'], color="orange", lw=1, label="Booby_Island")
f = ax.plot(Freemantle['Tide'], color="red", lw=1, label="Freemantle")
ax.set_xlabel("Date")
ax.set_ylabel("Water height (mm)")
ax.tick_params(axis='x', rotation=45)
ax.legend()
ax.set_xlim([datetime.date(2008, 1, 1), datetime.date(2008, 12, 31)])
fig_summary.tight_layout()

The plot, though busy, already shows some interesting features. Booby Island has the highest tidal range and shows a clear annual signal. Freemantle has the smallest tidal range and seems fairly constant.

Tide gauges often record the water level so can pick up storms and are affected by even small winds blowing onshore for example. This means the data you see might not be "just tides", but also some aspect of weather, depending on how the data are processed. In addition, for long term records, the tide gauges will also record sea level rise. All heights are measured above a datum so you also have to be careful comparing raw data from one tide gauge to another.

Let's now look at one month in detail:

In [ ]:
fig_june=plt.figure()
ax=fig_june.add_subplot(111)
fd = ax.plot(Fort_Denison['Tide'], color="blue", lw=1, label="Fort Denison")
bi = ax.plot(Booby_Island['Tide'], color="orange", lw=1, label="Booby_Island")
f = ax.plot(Freemantle['Tide'], color="red", lw=1, label="Freemantle")
ax.set_xlabel("Date")
ax.set_ylabel("Water height (mm)")
ax.tick_params(axis='x', rotation=45)
ax.legend()
ax.set_xlim([datetime.date(2008, 6, 1), datetime.date(2008, 7, 1)])
fig_june.tight_layout()

We can now see more of the tidal signals. You may notice Freemantle shows fewer wiggles than the other two locations; this is because it is a diurnal tide: one tide per day. Fort Denison has a clear two tides per day; semi-diurnal. Booby Island normally has two tides per days, but some days, those two tides effectively blend into one (see around the 17th June). This is a mixed diurnal-semi diurnal system.

All tidal signals are essentially a mix of multiple sine curves. We know what the frequency of the external forcing is for tides (the rotation of the earth, the movement of the moon around the earth, etc), which allows us to break up the signal into constituent parts: the tidal constituents. These are often given labels such as M2, S2, K1, O1, etc. Each one has a particular frequency as given in the table below.

Decription                        |	Darwin symbol | Period (h) | Speed (°/h)
----------------------------------|---------------|------------|------------
Principal lunar semidiurnal	      | M2            | 12.4206012 | 28.9841042
Principal solar semidiurnal	      | S2	          | 12         |	30	      
Lunar diurnal	                    | K1	          | 23.9344721 |	15.0410686
Lunar diurnal	                    | O1	          | 25.8193387 |	13.9430356
Lunisolar semidiurnal	            | K2	          |11.96723606 |	30.0821373
Larger lunar elliptic semidiurnal	| N2	          | 12.6583475 |	28.4397295

There are over 400 different contiuents, but 60-ish is considered enough for accurate tidal predictions, but fewer still can be used for useful tidal predictions. The tidal signal can then be reconstructed by summing sine curves of the correct frequency and amplitude (and phase) to recreate and then predict the tides. At each location the amplitude and phase will be different.

Let's add up some sine curves to make something like the tide signals above.


# Sine curves

Sine cuves can be generalised to the formula:

$y = A sin (Bx+C) + D$

$A$ is the amplitude, $2\pi/B$ is the period, $C$ is the phase shift and $D$ is the vertical shift. We know the period (from the table above!) and $D$ is not relevant here, so we have two parameters to play with: $A$ and $C$.

In [ ]:
A_m2 = 0.53
B_m2 = 12.4206012 # hours
C_m2 = 0

times = np.arange(0,24*14,0.5) # 14 days in hours
sin_curve = A_m2*np.sin(2*math.pi/B_m2*times + C_m2)

plt.plot(times,sin_curve)
plt.xlabel("Hours")
plt.ylabel("Water height (m)")

In [ ]:
# Let's now add an S2 curve on that
A_s2 = 0.23
B_s2 = 12
C_s2 = math.pi/2

sin_curve = A_m2*np.sin(2*math.pi/B_m2*times + C_m2) + \
            A_s2*np.sin(2*math.pi/B_s2*times + C_s2)
plt.plot(times,sin_curve)
plt.xlabel("Hours")
plt.ylabel("Water height (m)")

The above shows what in effect is spring and neap tide using just two components, with different amplitudes and phases (and frequencies/periods).

We can extract constituents from a tidal signal like those plotted above using least squares regression analysis to work out what the tidal constiuents are. Let's do that now to pull out the common consituents from our tidal data.

In [ ]:
# let's first pull out a single year's worth of data
# and remove the mean value so the tides oscillate across zero
FD_2008 = extract_single_year_remove_mean(2008, Fort_Denison)
BI_2008 = extract_single_year_remove_mean(2008, Booby_Island)
F_2008 = extract_single_year_remove_mean(2008, Freemantle)


In [ ]:
# We can use the module uptide to work out the tidal constiuents
# More on uptide: https://github.com/stephankramer/uptide
import uptide
# we create a Tides object with a list of the consituents we want.
tide = uptide.Tides(['M2'])
# We then set out start time. All data must then be in second since this time
tide.set_initial_time(datetime.datetime(2008,1,1,0,0,0))
# so let's swap our dates for seconds since midnight 1/1/2008.
# Note the 1e9 (the int64 seconds epoch in numpy is multiplied by this for some reason)
seconds_since = (FD_2008.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2008,1,1,0,0,0).timestamp()
# We then send the elevation data (our tides) and time in seconds to uptide
# and do the harmonic analysis
amp,pha = uptide.harmonic_analysis(tide, FD_2008['Tide'].to_numpy()/1000, seconds_since)

# uptide returns the amplitudes as a list (in the order of the constiuents listed above) and the phases (in radians)
print(amp, pha)


The above numbers are the tidal amplitude for the M2 constiuent at Fort Denison (0.5013 m) and the phase (5.366 radians). We can look up what the actual numbers are for Fort Denison (exercise for the reader!). However, we've forgotten something: *timezones*!

We also need to account for the timezone of the data. The phase is measured relative to UTC/GMT. We therefore need to tell the analysis the time data is in the Sydney timezone.

In [ ]:
tz = pytz.timezone("Australia/Sydney")
tide.set_initial_time(datetime.datetime(2008,1,1,0,0,0))
seconds_since = (FD_2008.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2008,1,1,0,0,0,tzinfo=tz).timestamp()

amp,pha = uptide.harmonic_analysis(tide, FD_2008['Tide'].to_numpy()/1000, seconds_since)
print(amp,pha)

We now have the phase, in radians, relative to UTC/GMT, but taking into account the timezone. In the code box below, write code, using the box above as a template, to get the amplitudes and phase for M2 only at Booby Island and Freemantle.

In [ ]:
# Write code to obtain the M2 component from Booby Island and Freemantle. Rember the timezone!
# tz = pytz.timezone("Australia/Perth")
# tz = pytz.timezone("Australia/Lindeman")

What happens when we want multiple constiuents? The first thing we need to consider is how long our data record is. Consituents that have a period/frequency that is close together need a longer dataset to be able to seperate them out. We can use something called the Rayleigh Critereon to work out how long a record we need.

In [ ]:
constituents  = ['M2', 'S2', 'N2', 'K2', 'O1', 'P1', 'Q1', 'M4']
print(uptide.select_constituents(constituents,15*24*60*60)) # This is 15 days in seconds

What we get back is that we can't resolve the N2, K2 and Q1 from the list with 15 days worth of data. What if we had 30 days?

In [ ]:
constituents  = ['M2', 'S2', 'N2', 'K2', 'O1', 'P1', 'Q1', 'M4']
print(uptide.select_constituents(constituents,30*24*60*60)) # This is 15 days in seconds

Nope! K2 is still not able to be resolved from 30 days worth of data. How many days would we need?

In [ ]:
tide = uptide.Tides(constituents)
print(tide.get_minimum_Rayleigh_period()/86400.)

182.6 days worth of data to be able to work out the constituents listed above. So with our year of data we should be fine!

In [ ]:
tz = pytz.timezone("Australia/Sydney")
tide.set_initial_time(datetime.datetime(2008,1,1,0,0,0))
seconds_since = (FD_2008.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2008,1,1,0,0,0,tzinfo=tz).timestamp()

amp,pha = uptide.harmonic_analysis(tide, FD_2008['Tide'].to_numpy()/1000, seconds_since)
print(amp, pha)

At Fort Denison our amplitudes and phases are:


*   M2: 0.503m and 4.182 radians
*   S2: 0.125m and 4.599 radians
*   N2: 0.115 and 3.93 radians
*   K2: 0.036 and 4.27 radians
*   O1: 0.097 and 1.42 radians
*   P1: 0.043 and 1.99 radians
*   Q1: 0.020 and 1.016 radians
*   M4: 0.003 and 2.09 radians

We could plot those using a bunch of sine curves and compare to our data...

Fortunately for us, `uptide` also contains functions to plot a theoretical tidal curve from a number of phases and amplitudes for given constiuents.





In [ ]:
t = np.arange(0, 365*24*3600, 1800) # 1 year in 1800 second intervals
eta = tide.from_amplitude_phase(amp, pha, t)
fig_summary=plt.figure()
ax=fig_summary.add_subplot(111)
# note we use seconds since as t (for the theoretical plot) is also in seconds
fd = ax.plot(seconds_since/86400, FD_2008['Tide']/1000, color="blue", lw=1, label="Fort Denison")
theoretical = ax.plot(t/86400, eta, color="orange", lw=1, label="Theoretical")
ax.set_xlabel("Days")
ax.set_ylabel("Water height (m)")
ax.tick_params(axis='x', rotation=45)
# uncomment line below and rerun to see a zoom in
#ax.set_xlim([14, 44]) # only plot 30 days worth
ax.legend()
fig_summary.tight_layout()

Exercise: Create similar plots for Booby Island and Freemantle in the semi-completed code blocks below.

In [ ]:
# Booby Island

# Analyse the tides and store in amp, pha

t = np.arange(0, 365*24*3600, 1800) # 1 year in 1800 second intervals
eta = tide.from_amplitude_phase(amp, pha, t)
fig_summary=plt.figure()
ax=fig_summary.add_subplot(111)
# add the res tof the plotting code

In [ ]:
# Freemantle

# Analyse the tides and store in amp, pha

t = np.arange(0, 365*24*3600, 1800) # 1 year in 1800 second intervals
eta = tide.from_amplitude_phase(amp, pha, t)
fig_summary=plt.figure()
ax=fig_summary.add_subplot(111)
# add the res tof the plotting code

Questions to think about:


1.   None of our theoretical tidal curves are perfect. Why?
2.   How would you go about predicting tides into the future?



# Tidal modelling

Whilst the tidal gauge analysis is useful (and widely used!) it is ony one dimensional. It will only give information at a single location through time. Tides are three dimensional: they also vary in space, which we can see by looking at different tide gauges. How do we fill in the space between tide gauges? Numerical modelling of the tides is the answer to that question! There are a large number of tidal models available that can simulate local, regional or global tides. Most use the Shallow Water Equations (SWE), which simulates the water surface moving up and down due to gravitational forcing and the flow of water that generates. SWE models have no vertical structure though; they simulate the ocean and seas as a single layer with a depth-averaged velocity. But because of that they are quick (relatively speaking) to run.

The big advantage of a numerical model is that you can ask hypothetical questions, such as "how will the tides change as sea-level rises?". A number of studies have done just that and found that changes in the tidal range in places will be up to 10% of the sea-level rise. This alters your flood risk at the coast by an additional 10% (either increase or decrease, but obviously a 10% increase is bad!).

We can't run a detailed model in Google Colab unfortunately, but we can have a look at the results from one and compare to tide gauges!

[In our recent paper on the future tides on the GBR](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2021JC017823), we ran tidal models for 90 simulated days (which took a few hundred cores a couple of weeks to run). We validated the model against tide gauges for the present day sea level, and then re-ran the model with 0.43 and 1.1 m of relative sea-level rise.

We're going to pull data from a station that's in our model and also grab data for that virtual tide gauge in the model and compare.

In [ ]:
townsville_url = "https://uhslc.soest.hawaii.edu/data/csv/fast/hourly/h334.csv"
model_townsville_url = "https://raw.githubusercontent.com/jhill1/tidal_analysis_workshop/main/Mawson_et_al_Townsville.csv"

file_name = os.path.basename(townsville_url) # get the full path to the file
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly
file_name = wget.download(townsville_url, out=".")

Townsville = read_and_process_data("h334.csv")

file_name = os.path.basename(model_townsville_url) # get the full path to the file
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly
file_name = wget.download(model_townsville_url, out=".")

# Load in model data and reformat to match the tide gauge data
Model_Townsville = pd.read_csv("Mawson_et_al_Townsville.csv")
Model_Townsville['Date'] = pd.to_datetime(Model_Townsville['Time (s)'],
                                          unit='s',
                                          origin=datetime.datetime(2000,1,1,0,0,0))
Model_Townsville = Model_Townsville.rename(columns={'Height (m)': 'Tide'})
Model_Townsville = Model_Townsville.drop('Time (s)', axis=1)
Model_Townsville = Model_Townsville.set_index('Date')
# remove first 5 days (spin-up of the model) to leave 85 days of data
Model_Townsville = Model_Townsville['2000-01-05':'2000-03-31']

# Our model ran from 1st Jan 2000, so let's pull the year 2000 from the real data
Townsville_2000 = extract_single_year_remove_mean(2000, Townsville)
Townsville_2000['Tide'] /= 1000

# restrict to 85 days to match model
Townsville_2000 = Townsville_2000['2000-01-05':'2000-03-31']


In [ ]:
fig_model=plt.figure()
ax=fig_model.add_subplot(111)
fd = ax.plot(Model_Townsville['Tide'], color="blue", lw=1, label="Model")
bi = ax.plot(Townsville_2000['Tide'], color="orange", lw=1, label="Tide Gauge")
ax.set_xlabel("Date")
ax.set_ylabel("Water height (mm)")
ax.tick_params(axis='x', rotation=45)
ax.legend()
fig_june.tight_layout()

Let's analyse the model data and measured data to see how good the model is. We can see from the graph above it's not perfect, so how close is it?

The model was forced with the following consituents: M2, S2, N2, K2, K1, O1, P1, Q1, M4, MS4 and MN4


In [ ]:
# check rayleigh criterion for which constiuents
constituents  = ['M2', 'S2', 'N2', 'K2', 'K1', 'O1', 'P1', 'Q1', 'M4', 'MS4', 'MN4']
print(uptide.select_constituents(constituents,85*24*60*60))


In [ ]:
# Note I've adjusted the order to pull out the more important consituents first,
# removing the ones we can't resolve
constituents  = ['M2', 'S2', 'K1', 'O1', 'N2', 'Q1', 'M4', 'MS4', 'MN4']
tide = uptide.Tides(constituents)
tide.set_initial_time(datetime.datetime(2000,1,1,0,0,0))
tz = pytz.timezone("Australia/Lindeman")
seconds_since = (Model_Townsville.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2000,1,1,0,0,0,tzinfo=tz).timestamp()

m_amp,m_pha = uptide.harmonic_analysis(tide, Model_Townsville['Tide'].to_numpy(), seconds_since)
# The amplitudes and pahses are printed in the same order as the constiuents listed above
print(constituents)
print(m_amp)
print(np.degrees(m_pha))

In [ ]:
# The above is the model; what about the data?
seconds_since = (Townsville_2000.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2000,1,1,0,0,0,tzinfo=tz).timestamp()

amp,pha = uptide.harmonic_analysis(tide, Townsville_2000['Tide'].to_numpy(), seconds_since)
print(constituents)
print(amp)
print(np.degrees(pha))

The Australian Hydrological Office has the following data for Townsville, which can compare to the model and a tide gauge:

Amplitudes:

Data |Q1 amp | O1 amp | K1 amp | N2 amp | M2 amp | S2 amp |
-----|-------|--------|--------|--------|--------|--------|
AHO  | 0.0296| 0.1638 | 0.3389 | 0.2374 | 0.7418 | 0.4268 |
Model| 0.0315| 0.1631 | 0.3331 | 0.2463 | 0.7808 | 0.4490 |
Gauge| 0.0306| 0.1724 | 0.3500 | 0.2232 | 0.7396 | 0.4276 |

Phases:

 Data    | Q1     | O1    | K1     | N2     | M2     | S2    |
---------|--------|-------|--------|--------|--------|-------|
AHO      | 126.73 | 152.17| 188.14 | 255.76 | 278.09 | 247.61|
Model    | 128.58 | 143.68| 193.40 | 242.15 | 264.93 | 244.98|
Gauge    | 121.55 | 148.07| 202.95 | 247.82 | 274.49 | 255.75|

You can see that the model performs well; in most cases being closer than the tide gauge data to the AHO constituents. Why would this be the case?


**One Tree Island**

The model also simulates One Tree where the UoS reasearch station is situated. Using the code above, can you extract the modelled tidal consituents for OTI? The code below will get you started, but use the code for Townsville, above, to finish it off.

In [ ]:
# Repeat above for OTI (but we have no long term tidal gauge station there)
model_oti_url = "https://raw.githubusercontent.com/jhill1/tidal_analysis_workshop/main/Mawson_et_al_OTI.csv"
file_name = os.path.basename(model_oti_url) # get the full path to the file
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly
file_name = wget.download(model_oti_url, out=".")

# Load in model data and reformat to match the tide gauge data
Model_OTI = pd.read_csv("Mawson_et_al_OTI.csv")

The AHO has the following data for OTI. How close is the model? Fill in your analysis results in the table below (double click this cell and complete the table).

Amplitudes:

Data |Q1 amp | O1 amp | K1 amp | N2 amp | M2 amp | S2 amp |
-----|-------|--------|--------|--------|--------|--------|
AHO  | 0.0264| 0.1554 | 0.2832 | 0.1905 | 0.923  | 0.3378 |
Model|       |        |        |        |        |        |


Phases:

 Data    | Q1     | O1    | K1     | N2     | M2     | S2    |
---------|--------|-------|--------|--------|--------|-------|
AHO      | 77.60  | 114.65| 153.43 | 219.90 | 248.08 | 254.41|
Model    |        |       |        |        |        |       |

Finally, we can plot the model against the theoretical tides, as we did above. Complete the code below, using the table above, to plot both the model and tide gauge reconstruction at OTI, and compare.

In [ ]:
t = np.arange(0, 85*24*3600, 1800) # 85 days in 1800 second intervals
# Set up our constiuents
constituents  = ['M2', 'S2', 'K1', 'O1', 'N2', 'Q1']
# add in two arrays for amplitude and phase
amp = [ ]
pha = [ ]
#
eta = tide.from_amplitude_phase(amp, pha, t)
fig_summary=plt.figure()
ax=fig_summary.add_subplot(111)
seconds_since = (Model_Townsville.index.astype('int64').to_numpy()/1e9) - datetime.datetime(2000,1,1,0,0,0,tzinfo=tz).timestamp()
# note we use seconds since as t (for the theoretical plot) is also in seconds
fd = ax.plot(seconds_since/86400, Model_OTI['Tide'], color="blue", lw=1, label="Model")
theoretical = ax.plot(t/86400, eta, color="orange", lw=1, label="Theoretical")
ax.set_xlabel("Days")
ax.set_ylabel("Water height (m)")
ax.tick_params(axis='x', rotation=45)
ax.legend()
fig_summary.tight_layout()

# Summary

This notbook goes through the basics of analysing tidal gauge data and comparing to virtual tide gauges from a model. Understanding this will help you interpret tidal gauge data, but also how complex a tidal model can be and how helpful they are.

[I've created a fully completed version of this notebook here](https://github.com/jhill1/tidal_analysis_workshop/blob/main/Tidal_analysis_answers.ipynb) if you need to find any answers or check your code against mine.

You can download a copy of the notebook or store to your google drive or github.